In [9]:
import cv2 
import numpy as np
from skimage.draw import line as line_points

map = cv2.imread('map.jpg', cv2.IMREAD_COLOR)

# set initial point 
# our diminsions are 400, 680
x = 100
y = 220

# check that we do not stand at a wall.
while map[x][y][0] < 30:
    x += 4
    y += 4


# use sklearn line
points = line_points(x, y, 200, 400)
print (points)

(array([100, 101, 101, 102, 102, 103, 103, 104, 104, 105, 106, 106, 107,
       107, 108, 108, 109, 109, 110, 111, 111, 112, 112, 113, 113, 114,
       114, 115, 116, 116, 117, 117, 118, 118, 119, 119, 120, 121, 121,
       122, 122, 123, 123, 124, 124, 125, 126, 126, 127, 127, 128, 128,
       129, 129, 130, 131, 131, 132, 132, 133, 133, 134, 134, 135, 136,
       136, 137, 137, 138, 138, 139, 139, 140, 141, 141, 142, 142, 143,
       143, 144, 144, 145, 146, 146, 147, 147, 148, 148, 149, 149, 150,
       151, 151, 152, 152, 153, 153, 154, 154, 155, 156, 156, 157, 157,
       158, 158, 159, 159, 160, 161, 161, 162, 162, 163, 163, 164, 164,
       165, 166, 166, 167, 167, 168, 168, 169, 169, 170, 171, 171, 172,
       172, 173, 173, 174, 174, 175, 176, 176, 177, 177, 178, 178, 179,
       179, 180, 181, 181, 182, 182, 183, 183, 184, 184, 185, 186, 186,
       187, 187, 188, 188, 189, 189, 190, 191, 191, 192, 192, 193, 193,
       194, 194, 195, 196, 196, 197, 197, 198, 198, 199, 199, 2

In [10]:


import cv2
from typing import Tuple,List
from math import sin,cos,pi
from skimage.draw import line as line_points
import numpy as np
from numpy.linalg import norm



In [11]:
WHITE = (255,255,255)
RED = (0,0,255)
GREEN = (0,255,0)
BLACK = (0,0,0)
ROBOT_BORDER=(6,6,6)
FILL = -1
ROBOT_RADIUS = 10
ROBOT_BORDER_THICKNESS = 2
DIRECTION_LINE_LENGTH = 20
RADIAN = pi/180.0
SENSOR_ANGLE = 250
SENSOR_RESOLUTION = 2

In [12]:
def xy_from_r_theta_xy(r:float,theta: float, x: int, y:int) -> Tuple[int,int]:
    '''
        Takes the starting point of a line, its slope angle and a length, and returns the ending point
    '''
    x2 = int(x + r * cos(theta *RADIAN))
    y2 = int(y + r * sin(theta *RADIAN))

    return x2,y2

def line(img: cv2.Mat,
         from_point: Tuple[int,int],
         with_length: float,
         at_angle : float,
         color: Tuple[int,int,int],
         thickness: int
         ) -> cv2.Mat:
    
    x,y = from_point
    x2,y2 = xy_from_r_theta_xy(with_length,at_angle,x,y)
    return cv2.line(img,from_point,(x2,y2),color,thickness)

def cast_ray(img: cv2.Mat,
             from_point: Tuple[int,int],
             with_max_length: int,
             at_angle: float,
             color: Tuple[int,int,int]) -> Tuple[int,int]:

    x1,y1 = from_point

    farthest_point = xy_from_r_theta_xy(with_max_length,at_angle,x1,y1)
    x2,y2 = farthest_point

    lpoints = list(zip(*line_points(x1,y1,x2,y2)))

    endpoint = None
    done = False
    for x,y in lpoints:
        # check if the difference between current pixel and black is less than 10
        # if norm(img[y,x] - np.array(BLACK)) < 10:
        if norm(img[y,x] - np.array(BLACK)) == 0 :
            done = True
            endpoint = (x,y)
            break
        if not done: endpoint = (x,y)
       
    
    img = cv2.line(img,(x1,y1),endpoint,color,thickness=1)
    return endpoint

def draw_robot(map, x, y, t):
    img = cv2.circle(map,
                    center= (x,y),
                    radius= ROBOT_RADIUS,
                    color = RED,
                    thickness=FILL)
    img = cv2.circle(img,
                    center= (x,y),
                    radius= ROBOT_RADIUS,
                    color = ROBOT_BORDER,
                    thickness=ROBOT_BORDER_THICKNESS)
    img = line(img,
                from_point= (x,y),
                with_length= DIRECTION_LINE_LENGTH,
                at_angle= t,
                color= ROBOT_BORDER,
                thickness= ROBOT_BORDER_THICKNESS)
    return img

def move_ray(distance: int, x: float, y: float, angle: int):
    return round(x + distance * np.cos(angle * np.pi / 180)), round(y + distance * np.sin(angle * np.pi / 180))

In [17]:
def requirement1(image_path: str , x: int, y: int, t: int):
    '''
        This function is for requirement 1

        Parameters:
            image_path: str
                Path to the image
            x,y: int
                Location of Robot, (0,0) is the left-top corner of the 2D map, so x increases to the right and y increases to the down.
            t: int
                Angle of Robot's pose, 0 means robot is facing to the right, 90 means to the down, 180 to the left, 270 to the up.
    '''
    # Reading an image in color mode
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # write the position of the robot in txt file
    file = open("measurements.txt", "w")
    file.write("Robot position: " + "x: " + str(x) + " y: " + str(y) + " theta: " + str(t))

    # draw the robot
    image = draw_robot(image, x, y, t)

    # create array to save the distance at each angle
    distances = np.zeros(int(SENSOR_ANGLE/SENSOR_RESOLUTION)+1)
    endpoints = []

    index = 0

    # write meaeurements in txt file
    file.write("\n\nMeasurements: ") 
    file.write("\nAngle\tDistance\tEndpoint")


    for angle in range(t- int(SENSOR_ANGLE/ 2),t+ int(SENSOR_ANGLE/2) + 1,2):
        endpoint = cast_ray(image,
                            from_point= (x,y),
                            with_max_length= int(12 *100 / 4),
                            at_angle= angle,
                            color= GREEN)
                            

        # calculate the distance between the robot and the endpoint
        distance = norm(np.array((x,y)) - np.array(endpoint))

        # save the distance in the array
        distances[index] = min(1200, distance * 4)
        endpoints.append(endpoint)

        # write meaeurements in txt file
        file.write("\n" + str(angle) + "\t" + str(distances[index]) + "\t" + str(endpoint))
        
        # increase the index
        index += 1
    
    file.close()

    cv2.imwrite('req1.jpg', image)

    return distances , np.array(endpoints) 


requirement1('map.jpg', 220, 240, 120)

(array([ 116.6190379 ,  116.27553483,  116.06894503,  116.        ,
         116.06894503,  116.6190379 ,  117.09824935,  117.71151176,
         118.45674316,  119.33147112,  120.33287165,  121.45781161,
         122.7028932 ,  124.06449935,  125.53883861,  127.12198866,
         130.59862174,  132.48396129,  134.46189051,  138.67948659,
         140.91131963,  145.60219779,  148.05404419,  153.15351775,
         158.49290205,  164.04877324,  169.79988221,  175.72706109,
         184.9107893 ,  191.20669444,  200.87807247,  210.78899402,
         224.32119828,  238.15961035,  252.25384041,  273.78823934,
         295.70255325,  321.64576789,  355.46026501,  393.48697564,
         443.44108966,  513.27965087,  599.33296255,  639.86248523,
         636.55321851,  634.47616188,  632.80960802,  632.11391378,
         632.11391378,  632.80960802,  634.47616188,  637.04316965,
         639.86248523,  644.04968752,  649.09783546,  654.98702277,
         661.69479369,  669.19653316,  124.06449